In [40]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/text-similarity/train.csv
/kaggle/input/text-similarity/test.csv


In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from gensim.similarities import WmdSimilarity
from nltk import word_tokenize
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

In [42]:
train = pd.read_csv('/kaggle/input/text-similarity/train.csv')
test = pd.read_csv('/kaggle/input/text-similarity/test.csv')

In [43]:
# Remove stopwords.
stop_words = stopwords.words('english')
x_list = []
y_list = []
x_list = [w.lower() for w in train['description_x'] if w not in stop_words]
y_list = [w.lower() for w in train['description_y'] if w not in stop_words]
test_x_list = []
test_y_list = []
test_x_list = [w.lower() for w in train['description_x'] if w not in stop_words]
test_y_list = [w.lower() for w in train['description_y'] if w not in stop_words]

In [44]:
train.head()
test.head()
x_list
y_list
z_list = x_list + y_list
z_list

,Unnamed: 0,description_x,description_y,ticker_x,ticker_y,same_security
0,0,first trust dow jones internet,first trust dj internet idx,FDN,FDN,True
1,1,schwab intl large company index etf,schwab strategic tr fundamental intl large co ...,FNDF,FNDF,True
2,2,vanguard small cap index adm,vanguard small-cap index fund inst,VSMAX,VSCIX,False
3,3,duke energy corp new com new isin #us4 sedol #...,duke energy corp new com new isin #us26441c204...,DUK,DUK,True
4,4,visa inc class a,visa inc.,V,V,True


,test_id,description_x,description_y,same_security
0,0,semtech corp,semtech corporation,NaN
1,1,vanguard mid cap index,vanguard midcap index - a,NaN
2,2,spdr gold trust gold shares,spdr gold trust spdr gold shares,NaN
3,3,vanguard total bond index adm,vanguard total bond market index,NaN
4,4,oakmark international fund class i,oakmark international cl i,NaN


['first trust dow jones internet',
 'schwab intl large company index etf',
 'vanguard small cap index adm',
 'duke energy corp new com new isin #us4 sedol #b7jzsk0',
 'visa inc class a',
 'ford motor co new div: 0.600',
 'united states steel corp',
 'vanguard total international bond index etf',
 'schwab strategic tr us sml c',
 'mfs value fd cl i',
 'oppenheimer dev markets class a',
 'genomic health',
 'spdr barclays international etf',
 'kraft heinz company',
 'vanguard total bond market index fund admiral shares',
 'vanguard intermediate-term investment-grade fund investor shares',
 'baidu inc spon adr',
 'novartis ag- reg',
 'seaspan corp',
 'vanguard total bond market index admiral',
 'mfs muni high income fd cl i',
 'metlife inc com',
 'russell gbl real estate securities cl s',
 'vanguard short-term investment-grade fund admiral shares',
 'spdr s&p 500 etf trust',
 'pimco total return a',
 'intel corp',
 'vanguard 500 index fund - admiral class',
 'paypal holdings inc',
 'pim to

['first trust dj internet idx',
 'schwab strategic tr fundamental intl large co index etf',
 'vanguard small-cap index fund inst',
 'duke energy corp new com new isin #us26441c2044 sedol #b7jzs',
 'visa inc.',
 'ford motor co',
 'united sts stl cp (new)',
 'vanguard total intl bond index etf',
 'schwab us small cap etf',
 'mfs value fund cl i',
 'oppenheimer developing mkts fd cl a',
 'genomic health inc com',
 'spdr barclays capital international trea',
 'kraft heinz co com',
 'vanguard total bond market idx instl pls',
 'vanguard interm-term invmt-grade inv',
 'baidu inc spon adr rep a',
 'novartis ag',
 'seaspan corporation cmn',
 'vanguard bond index total mkt investor',
 'mfs municipal high cl i',
 'metlife incorporated',
 'vy clarion real estate port srv',
 'vanguard short-term invest-grade',
 'spdr s&p 500 etf iv',
 'fund: pimco total return admin',
 'intel corporation',
 'vanguard 500 idx adm',
 'paypal holdings inco',
 'pimco total return fund - class r',
 'vanguard small cap 

['first trust dow jones internet',
 'schwab intl large company index etf',
 'vanguard small cap index adm',
 'duke energy corp new com new isin #us4 sedol #b7jzsk0',
 'visa inc class a',
 'ford motor co new div: 0.600',
 'united states steel corp',
 'vanguard total international bond index etf',
 'schwab strategic tr us sml c',
 'mfs value fd cl i',
 'oppenheimer dev markets class a',
 'genomic health',
 'spdr barclays international etf',
 'kraft heinz company',
 'vanguard total bond market index fund admiral shares',
 'vanguard intermediate-term investment-grade fund investor shares',
 'baidu inc spon adr',
 'novartis ag- reg',
 'seaspan corp',
 'vanguard total bond market index admiral',
 'mfs muni high income fd cl i',
 'metlife inc com',
 'russell gbl real estate securities cl s',
 'vanguard short-term investment-grade fund admiral shares',
 'spdr s&p 500 etf trust',
 'pimco total return a',
 'intel corp',
 'vanguard 500 index fund - admiral class',
 'paypal holdings inc',
 'pim to

In [45]:
model = Word2Vec(z_list, min_count=1,size= 200,workers=3, window =3, sg = 1)

In [46]:
model.init_sims(replace=True)  # Normalizes the vectors in the word2vec class.
distance = model.wmdistance(test_x_list[0], test_y_list[0])
print('distance = %.4f' % distance)

distance = 0.1244


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).
  
